## Principal Component Analysis
We briefly mentioned a few naive approaches to manipulating the number of features used for describing a problem. Another approach would be to combine features by linear (or later even nonlinear transformations) in a more methodical way, that respects the *correlations* in a dataset. The idea is visualized in the graph below, where two features $x_1$ and $x_2$ are correlated:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

x = np.random.rand(40)
y = -0.3 + 0.5*x
y_test = -0.3 + 0.5*x + 0.2*np.random.rand(40)-0.1
p1 = np.array([0.0, -0.3])
p2 = np.array([1.0,  0.2])


points = np.c_[x,y_test]

d=np.cross(p2-p1,points-p1)/np.linalg.norm(p2-p1)

dires = np.array([-d[i]*np.array([1/np.sqrt(2), -1/np.sqrt(2)]) for i in range(len(d))])
pps = points-dires
    
def plot_corr(line, dists):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111)

    #ax.set_xlim([])
    ax.set_ylim([-0.4, 0.4])
    
    ax.set_xlabel(r"$x_1$", fontsize=20)
    ax.set_ylabel(r"$x_2$", fontsize=20)
    
    ax.scatter(x, y_test)
    if line:
        ax.plot(x, y, lw=4, zorder=-20)
    if dists:
        for i in range(d.shape[0]):
            ax.plot([points[i,0], pps[i,0]], [points[i,1], pps[i,1]], \
                      color='darkorange', lw=3, alpha=0.7, zorder=-10)
        ax.scatter(pps[:,0], pps[:,1], marker='x')
    #plt.axes().set_aspect("auto")#1./plt.axes().get_data_ratio())
    ax.set_aspect("auto")
    
interact(plot_corr, line=False, dists=False)

interactive(children=(Checkbox(value=False, description='line'), Checkbox(value=False, description='dists'), O…

<function __main__.plot_corr(line, dists)>



It is clear that there is a linear trend in the data (the features are correlated) and that we can find a new set of axes (activate line), where the distances of all the points to the new $x^\prime$-axis are minimized, as in the image above when you activate dists.

These kinds of images are correlation plots. Sometimes manually deciding which features to eliminate or combine is possible by examining these correlation plots, but for hundreds of features this isn't really feasible. A technique that performs this combination of features in a way that reduces the overall number of features is principal component analysis (PCA). It tells us which features are the most important and provides the new axes like in the image above. PCA strives to maximize variance, while minimizing covariance of a dataset, so it finds rotated axes that best represent the dataset.

In the example above, the distances of the points to the new $x^\prime$-axis are very small, so to reduce the dimension of the problem, instead of providing the coordinates of each point for both axes, we could provide the projection of the points onto it and ignore the distance in $y^\prime$-direction, since it is very small.

Mathematically, PCA solves the following problem: For a given dataset $X$ with $N$ samples $x_i$ comprised of $F$ features ($X \in \mathbb{R}^{N\times F}$), find an $K\times F$-matrix $\mathbf R$ such that

\begin{equation*}
  \mathbf x_i^\prime = \mathbf R^\text{T} \mathbf x_i 
\end{equation*}

where $\mathbf x_i^\prime \in \mathbb{R}^K$ and $K \leq N$. Let's look at another example:


In [2]:
x = np.random.rand(100)
y = -0.3 + 0.5*x
y_test = -0.3 + 0.5*x + 0.5*np.random.rand(100)-0.25
p1 = np.array([0.0, -0.3])
p2 = np.array([1.0,  0.2])

points = np.c_[x,y_test]

d=np.cross(p2-p1,points-p1)/np.linalg.norm(p2-p1)

dires = np.array([-d[i]*np.array([0.701, -0.701]) for i in range(len(d))])
pps = points-dires
    
def plot_corr(line, dists, new_axes):
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    #plt.cla()
    
    ax.set_xlabel(r"$x_1$", fontsize=16)
    ax.set_ylabel(r"$x_2$", fontsize=16)
    
    ax.set_xlim([-0.3, 1.3])
    ax.set_ylim([-0.5, 0.6])
    
    ax.scatter(x, y_test)
            
    if line:
        ax.plot(x, y, lw=4, zorder=-20)
    if dists:
        for i in range(d.shape[0]):
            ax.plot([points[i,0], pps[i,0]], [points[i,1], pps[i,1]], \
                      color='darkorange', lw=3, alpha=0.7, zorder=-10)
        ax.scatter(pps[:,0], pps[:,1], marker='x')
    if new_axes:
        ax.arrow(0, -0.3, 1, 0.5, head_width=0.05, head_length=0.1, \
         fc='red', ec='red', length_includes_head=True, zorder=1000)
        ax.arrow(0, -0.3, -0.701*0.1, 0.701*0.1, head_width=0.02, head_length=0.05, \
         fc='red', ec='red', length_includes_head=True, zorder=1000)
        #plt.quiver(0, -0.3, [1, -0.701], [0.5, 0.701], color = 'red', lw = 3, zorder=1000)
    
    
interact(plot_corr, line=False, dists=False, new_axes=False)

interactive(children=(Checkbox(value=False, description='line'), Checkbox(value=False, description='dists'), C…

<function __main__.plot_corr(line, dists, new_axes)>

A lot of the information in the dataset is captured along the rotated $x^\prime$-axis, which is the axis along which the variance is maximal. The $y^\prime$-axis is orthogonal to this axis, maximizing the variance "perpendicular" to the first axis. If we project the points onto the $x^\prime$-axis, we can retain the most information about the original dataset. To accurately do all of this, we need the directions of the axes $x^\prime$ and $y^\prime$, as well as their "lengths" $\lambda_1$ and $\lambda_2$. The length of an axis is the variance of the data along this direction. In this sense, PCA fits a hyperellipsoid to the data.

### The Algorithm

To perform a PCA, the data has to be zero-centered first, which means that the mean of each feature over the whole dataset is calculated and subtracted from each respective column of every sample. The mean of feature $f$ is calculated as

\begin{equation*}
  \mu_f = \frac 1 N \sum_i^N X_{if}
\end{equation*}

$\mu$ is a vector containing the feature means as columns. The zero-centered sample matrix is then 

\begin{equation*}
  \bar{X} = X - U^\text{T}
\end{equation*}

where $U = \begin{bmatrix} \rule[-1ex]{0.5pt}{2.5ex} & \rule[-1ex]{0.5pt}{2.5ex} & \cdots & \rule[-1ex]{0.5pt}{2.5ex} \\ \mu & \mu & \cdots & \mu \\ \rule[-1ex]{0.5pt}{2.5ex} & \rule[-1ex]{0.5pt}{2.5ex} & \cdots & \rule[-1ex]{0.5pt}{2.5ex} \end{bmatrix}$. The next step is to calculate the covariance matrix $C$ of the data:

\begin{equation*}
  C = \frac 1 N \bar{X}^\text{T}\bar{X}
\end{equation*}

Recall that the $F\times F$ covariance matrix contains the variance of a variable on the diagonal, and the covariances on the off-diagonal elements. When we're trying to maximize variance (or, equivalently, minimize covariance), we're looking for a diagonal matrix. This is the last step; diagonalize the covariance matrix (which is possible, since $C$ is symmetric). This is done by finding the eigenvectors composing the matrix $V$ (more specifically, the *right* eigenvectors), such that

\begin{equation*}
  C = VDV^\text{T}
\end{equation*}

where $D$ is a diagonal matrix. The eigenvectors $v_i$ are the **principal components**. In this eigensystem, the entries $\lambda_i$ of $D$ are the **variance** of each data point in that system. In general, the eigenvalues and eigenvectors aren't sorted. After sorting them in decreasing order of magnitude of the eigenvalues, the reduction process is performed by truncating the last $F-K$ rows of $V$ and $D$, such that only the entries with the lowest variances are discarded:

\begin{equation*}
  R = V[0:K]
\end{equation*}

This is the reduction matrix $R$ from the problem description. Multiplying the full dataset by this reduction matrix gives the new dataset 

\begin{equation*}
  X^\prime = \bar{X}R \in \mathbb{R}^{N\times K}
\end{equation*}

with a reduced number $K$ of features. The first row of this dataset contains the projection of the first sample in the original dataset onto the first principal component, and so on. To get the embedding of the original data into this lower-dimensional subspace, the means $U$ have to be added again to complete the transformation.

### PCA by SVD

In practice, calculating and diagonalizing $C$ is expensive and a much easier method is to use *singular value decomposition* on the data matrix itself:

\begin{equation*}
  X = USV^\text{T}
\end{equation*}

such that 

\begin{equation*}
  C = \frac 1 N X^\text{T}X = \frac 1 N VS^\text{T}U^\text{T}USV^\text{T} = \frac 1 N V S^2 V^\text{T}
\end{equation*}

Comparing this to the result from above, we see that the right-singular eigenvectors in $V$ are the principal directions, and the variances are $\lambda_i = \frac{s_i^2}{N}$. You can see this by multiplying the equation above with $V$ from the right.<br />
The columns of $US$ are the principal components. Truncating all matrices like we saw in the last lesson to the first $K$ entries to compute 

\begin{equation*}
  X^\prime = U[0:K]S[0:K]V[0:K]^\text{T}
\end{equation*}

gives the *reduced* representation of the data, which now only consists of the $K$ most important feature combinations regarding the variance they explain for the original dataset.

### Summary
PCA is extremely common in dimensionality reduction. It can be used as a way to visualize a very high-dimensional dataset in 2D or 3D. See for example the 1D, 2D, and 3D PCA representation of *Andersen's Iris dataset* we've briefly seen in lecture 01, but is also used as a preprocessing step for machine learning algorithms. As we've seen in the last lesson about SVD, it can be applied to images to retain only the most important information about an image to reduce the data size and to be able to use much smaller deep learning models. It improves the performance of many machine learning (or classical) algorithms, since it minimizes correlations between the new axes and, if desired, removes unwanted features which are most likely noise in the dataset. The newly found axes are still orthogonal. In image-processing, this transformation is sometimes also called the *Karhunen-Loève transformation*.

PCA can be generalized as principal curve analysis or principal surface analysis, and very commonly *kernel-PCA* which is a nonlinear version of PCA using the kernel-trick.